In [1]:
# ==============================================================================
# CÉLULA 1: INSTALAÇÃO DE DEPENDÊNCIAS E AUTENTICAÇÃO
# ==============================================================================

print("Instalando dependências...")
!pip install earthengine-api --quiet
!pip install geopandas --quiet
!pip install rasterio --quiet
!pip install shap --quiet
!pip install geemap --quiet
print("Dependências instaladas.")

# ------------------------------------------------------------------------------
# Autenticação e Inicialização de Serviços Google
# ------------------------------------------------------------------------------
from google.colab import drive
import ee

# A autenticação com o Earth Engine pedirá um token de autorização.
try:
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine já inicializada.")
except Exception as e:
    print("Autenticando na API do Google Earth Engine...")
    ee.Authenticate()
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine inicializada.")

# Monta o Google Drive para que possamos salvar arquivos (datasets, modelos)
print("Montando Google Drive...")
drive.mount('/content/drive')
print("Google Drive montado em /content/drive")

Instalando dependências...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.8 MB/s eta 0:00:00
Dependências instaladas.
Autenticando na API do Google Earth Engine...
API do Google Earth Engine inicializada.
Montando Google Drive...
Mounted at /content/drive
Google Drive montado em /content/drive


In [2]:
# ==============================================================================
# CÉLULA 2: IMPORTAÇÃO DAS BIBLIOTECAS PRINCIPAIS
# ==============================================================================
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import torch

print("Bibliotecas principais importadas. Ambiente pronto para uso!")

Bibliotecas principais importadas. Ambiente pronto para uso!


In [3]:
# ==============================================================================
# PASSO 1: DEFINIÇÃO DOS PARÂMETROS GLOBAIS
# ==============================================================================

# 1. Período de Tempo para Análise(10 anos)
START_DATE = '2014-01-01'
END_DATE = '2024-01-01'

# 2. Área de Interesse(AOI)
# Coleção de biomas do Brasil carregada e filtrada para obter apenas o polígono do Cerrado.
# Este polígono 'cerrado_aoi' será usado para "recortar" todos os outros dados.
biomas = ee.FeatureCollection('projects/mapbiomas-workspace/AUXILIAR/ESTATISTICAS/COLECAO8/VERSAO-1/refined_biome')
cerrado_aoi = biomas.filter(ee.Filter.eq('NAME_PT_BR', 'Cerrado')).geometry()

# 3. Resolução Espacial(30m)
TARGET_RESOLUTION = 30

print("Parâmetros definidos:")
print(f"Período de análise: {START_DATE} a {END_DATE}")
print("Área de Interesse: Bioma Cerrado")

Parâmetros definidos:
Período de análise: 2014-01-01 a 2024-01-01
Área de Interesse: Bioma Cerrado


In [21]:
# ==============================================================================
# PASSO 2: COLETA DA ÁREA QUEIMADA (MAPBIOMAS)
# ==============================================================================

# 1. Carregamos o asset global como uma única ee.Image, SEM NENHUM CORTE.
burned_area_image_global = ee.Image('projects/mapbiomas-public/assets/brazil/fire/collection4/mapbiomas_fire_collection4_monthly_burned_v1')

# --- TESTE DE VERIFICAÇÃO SEGURO ---
# Agora fazemos as perguntas ao objeto global e simples.
print("Iniciando verificação segura no objeto global...")

# Nomes de todas as bandas do objeto global.
band_names_server = burned_area_image_global.bandNames()

# Pedimos ao servidor para nos enviar o NÚMERO TOTAL de bandas.
num_bands = band_names_server.size().getInfo()
print(f"O objeto global é uma imagem única com {num_bands} bandas.")

# Pedimos ao servidor para "fatiar" a lista e nos enviar APENAS OS 5 PRIMEIROS NOMES.
last_5_bands = band_names_server.slice(35, 40).getInfo()
print("Exemplo dos ultimos 5 nomes de banda:", last_5_bands)
print("Verificação concluída com sucesso.")

# ------------------------------------------------------------------------------

# 2. Objeto cortado para uso futuro
burned_area_cerrado = burned_area_image_global.clip(cerrado_aoi)

print("\nObjeto 'burned_area_cerrado' (recortado para o Cerrado) criado para uso futuro.")

Iniciando verificação segura no objeto global...
O objeto global é uma imagem única com 40 bandas.
Exemplo dos ultimos 5 nomes de banda: ['burned_monthly_2020', 'burned_monthly_2021', 'burned_monthly_2022', 'burned_monthly_2023', 'burned_monthly_2024']
Verificação concluída com sucesso.

Objeto 'burned_area_cerrado' (recortado para o Cerrado) criado para uso futuro.


In [20]:
# =============================================================================
# PASSO 3: GERAÇÃO DOS PONTOS DE IGNIÇÃO (RÓTULOS)
# =============================================================================

# --- 1. Selecionar o ano de interesse ---
YEAR_TO_PROCESS = 2022
band_name = f'burned_monthly_{YEAR_TO_PROCESS}'
burned_area_2022 = burned_area_cerrado.select(band_name)

# --- 2. Identificar as manchas de incêndio --- (Sem mudanças)
print("Identificando manchas de área queimada (MapBiomas)...")
fire_mask = burned_area_2022.gt(0)
fire_patches_image = fire_mask.connectedComponents(ee.Kernel.plus(1), 1024)

# --- 3. Vetorizar as manchas ---
print("Vetorizando manchas de área queimada...")
fire_polygons = fire_patches_image.select('labels').reduceToVectors(
    geometry=cerrado_aoi,
    scale=TARGET_RESOLUTION,
    geometryType='polygon',
    eightConnected=False,
    labelProperty='patch_id'
)

# --- 4. COLETA DOS FOCOS DE CALOR(MCD14ML)---
print("Carregando focos de calor do MCD14ML (Produto de Arquivo)...")
active_fires_2022_raw = ee.FeatureCollection("MODIS/C6/MCD14ML") \
    .filterDate(f'{YEAR_TO_PROCESS}-01-01', f'{YEAR_TO_PROCESS}-12-31') \
    .filterBounds(cerrado_aoi)

# Otimização: Selecionar apenas os dados que vamos usar
# NOTA: Os nomes das propriedades são diferentes! (ex: 'FRP', 'confidence' é número)
active_fires_2022_raw = active_fires_2022_raw.select(['acq_time', 'confidence', 'FRP'])
# 'acq_date' é uma propriedade do sistema (system:time_start)

# --- 5. Definir a Junção Espacial --- (Sem mudanças)
spatial_filter = ee.Filter.withinDistance(distance=0, rightField='.geo', leftField='.geo')
save_all_join = ee.Join.saveAll(matchesKey='matches')

# --- 6. Aplicar a junção ---
print("Aplicando junção espacial (Polígonos x Pontos)...")
intersecting_polygons = save_all_join.apply(fire_polygons, active_fires_2022_raw, spatial_filter)

# --- 7. Mapear para encontrar a primeira ignição ---
print("Mapeando resultados para encontrar primeira ignição...")
def find_first_ignition(polygon_with_matches):
    matched_points = ee.FeatureCollection(ee.List(polygon_with_matches.get('matches')))

    # Ordenamos pela data (timestamp) e depois pela hora (HHMM)
    first_point = matched_points.sort('system:time_start').sort('acq_time').first()
    return first_point.copyProperties(polygon_with_matches)

# --- 8. Formatar os resultados ---
# Esta função foi ATUALIZADA para os nomes de propriedade do MCD14ML
def format_ignition_point(feature):
    # 'acq_date' vem do timestamp do sistema
    date = ee.Date(feature.get('system:time_start')).format('YYYY-MM-DD')

    # 'acq_time' é uma string HHMM, formatamos para 'HH:mm'
    time_str = ee.String(feature.get('acq_time'))
    time = time_str.slice(0, 2).cat(':').cat(time_str.slice(2, 4))

    return ee.Feature(
        feature.geometry(),
        {
            'acq_date': date,                       # String 'YYYY-MM-DD'
            'acq_time': time,                       # String 'HH:mm'
            'confidence': feature.get('confidence'),  # Número 0-100
            'frp': feature.get('FRP')               # Número (MW) (NOTA: 'FRP' maiúsculo)
        }
    )

ignition_points_raw = intersecting_polygons.map(find_first_ignition, dropNulls=True)
ignition_points_2022 = ignition_points_raw.map(format_ignition_point)

print(f"\nCélula PASSO 2.1.3 (revisada com MCD14ML) executada com sucesso.")
print("A variável 'ignition_points_2022' está pronta para exportar com todos os dados corretos.")

Identificando manchas de área queimada (MapBiomas)...
Vetorizando manchas de área queimada...
Carregando focos de calor do MCD14ML (Produto de Arquivo)...
Aplicando junção espacial (Polígonos x Pontos)...
Mapeando resultados para encontrar primeira ignição...

Célula PASSO 2.1.3 (revisada com MCD14ML) executada com sucesso.
A variável 'ignition_points_2022' está pronta para exportar com todos os dados corretos.


In [15]:
# ==============================================================================
# PASSO 4: CARREGAMENTO DAS COLEÇÕES DE FEATURES
# ==============================================================================

# --- 4.1 DADOS METEOROLÓGICOS (ERA5-Land) ---
# Contém dados de temperatura, umidade, vento, etc.
era5_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
   .filterDate(START_DATE, END_DATE) \
   .filterBounds(cerrado_aoi)

print(f"Encontradas {era5_collection.size().getInfo()} imagens horárias de meteorologia.")

# --- 4.2 DADOS DE VEGETAÇÃO (MODIS & Landsat) ---
# NDVI/EVI (saúde da vegetação) e LST (temperatura da superfície) do MODIS.
ndvi_collection = ee.ImageCollection('MODIS/061/MOD13A2') \
   .filterDate(START_DATE, END_DATE) \
   .filterBounds(cerrado_aoi) \
   .select('NDVI')

lst_collection = ee.ImageCollection('MODIS/061/MOD11A1') \
   .filterDate(START_DATE, END_DATE) \
   .filterBounds(cerrado_aoi) \
   .select('LST_Day_1km')

print(f"Encontradas {ndvi_collection.size().getInfo()} imagens de NDVI (16 dias).")
print(f"Encontradas {lst_collection.size().getInfo()} imagens de LST (diárias).")

# --- 4.3 DADOS TOPOGRÁFICOS (SRTM) ---
# Elevação é uma imagem única (estática), não uma coleção.
elevation = ee.Image('USGS/SRTMGL1_003').clip(cerrado_aoi)
# Cálculo do declive (slope) diretamente a partir da elevação.
slope = ee.Terrain.slope(elevation)

print("Dados de elevação e declive carregados.")

# --- 4.4 DADOS ANTROPOGÊNICOS (MapBiomas) ---
# Uso e cobertura do solo.
land_cover = ee.Image('projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_coverage_v2').clip(cerrado_aoi)

print("Dados de uso e cobertura do solo carregados.")

Encontradas 87648 imagens horárias de meteorologia.
Encontradas 230 imagens de NDVI (16 dias).
Encontradas 3631 imagens de LST (diárias).
Dados de elevação e declive carregados.
Dados de uso e cobertura do solo carregados.


In [ ]:
# ==============================================================================
# PASSO 5: EXPORTAR RÓTULOS PARA O DRIVE
# ==============================================================================

# Define o nome do arquivo e a tarefa de exportação.
output_filename = f'ignition_points_cerrado_{YEAR_TO_PROCESS}'
task = ee.batch.Export.table.toDrive(
    collection=ignition_points_2022,
    description=output_filename,
    folder='Projeto_Incendios/data/raw',  # A pasta que criamos no seu Google Drive
    fileNamePrefix=output_filename,
    fileFormat='CSV'
)

# Inicia a tarefa de exportação.
task.start()

print("Tarefa de exportação iniciada. Monitore o status na aba 'Tasks' do Google Earth Engine.")
print(f"O arquivo '{output_filename}.csv' será salvo na pasta 'Projeto_Incendios/data/raw' do seu Google Drive.")